In [344]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats 
import pickle
from sklearn.externals import joblib

input_df = pd.read_csv('C:/Users/User/Desktop/sigmoid.csv')
# train_df = input_df[(input_df['bias'] > 0)&(input_df['bias'] <= 2)]
train_df = input_df[(input_df['yearmonth'] != 202007)&(input_df['bias'] > 0)&(input_df['bias'] <= 2)]
test_df  = input_df[(input_df['yearmonth'] == 202007)&(input_df['bias'] >= 0)&(input_df['bias'] <= 2)]

# create some exponential data
X =  np.array(train_df['bias'].values)[:,np.newaxis]
Y =  np.array(train_df['accuracy'].values/100)

test_X = np.array(test_df['bias'].values)[:,np.newaxis]
test_y = np.array(test_df['accuracy'].values/100)

# exponential
exp_x = np.exp(X)
exp_y = np.exp(Y)

# log scale
ln_X = np.log(X)
ln_X = np.nan_to_num(ln_X)
ln_y = np.log(Y)
ln_y = np.nan_to_num(ln_y)

#regression
lr = LinearRegression()
pr = LinearRegression()

#다항 회귀를 위한 2차항 추가
quadratic = PolynomialFeatures(degree=2)

def evaluation(X,Y,cat):
    X_quad = quadratic.fit_transform(X)
    
    #비교를 위한 단순선형회귀
    lr.fit(X,Y)
    X_fit = np.arange(min(X),max(X),0.1)[:,np.newaxis]
    y_lin_fit = lr.predict(X_fit)

    pr.fit(X_quad,Y)
    y_quad_fit = pr.predict(quadratic.fit_transform(X_fit))
    
##     quad피팅된 모델 저장 
#     model_name = cat+'model.pkl'
#     save_model = pickle.dumps(pr)
#     clf_from_pickle = pickle.loads(save_model)
#     joblib.dump(pr,model_name)
    
    y_lin_pred = lr.predict(X)
    y_quad_pred = pr.predict(X_quad)

    mse_lin = mean_squared_error(Y,y_lin_pred)
    mse_quad = mean_squared_error(Y,y_quad_pred)

    r2_lin = r2_score(Y,y_lin_pred)
    r2_quad = r2_score(Y,y_quad_pred)

#     print("\n MSE \ Linear : %.2f,\t Quadratic : %.2f"%(mse_lin,mse_quad))
    print("\n R2 : Linear : %.2f,\t Quadratic : %.2f"%(r2_lin,r2_quad))


    plt.scatter(X,Y,label="Data ")
    plt.plot(X_fit,y_lin_fit, label="linear fit", linestyle = '--')
    plt.plot(X_fit,y_quad_fit,label="quadratic fit")
    plt.legend(loc=2)
    plt.show()
    
#     if cat == 'original':
#         for i in range(0,50):
#             print('X : %.2f | Y : %.2f | Y_lin_pred : %.2f | Y_quad_pred : %.2f '%(X[i],Y[i],y_lin_pred[i],y_quad_pred[i]))
#     else:
#         for i in range(0,50):
#             print('X : %.2f | Y : %.2f | Y_lin_pred : %.2f | Y_quad_pred : %.2f '%(X[i],np.exp(Y[i]),np.exp(y_lin_pred[i]),np.exp(y_quad_pred[i])))    
    
    return pd.DataFrame()
# y_lin_fit = lr.predict(X)


def predict(X,Y,model):
    X_quad = quadratic.fit_transform(X)
#     print(X_quad)
    clf_from_joblib = joblib.load(model) 
    y_pred = clf_from_joblib.predict(X_quad)
    print('a0 : ',round(clf_from_joblib.intercept_,2))
    print('a1 : ',round(clf_from_joblib.coef_[1],4))
    print('a2 : ',round(clf_from_joblib.coef_[2],4))
    
    
    return y_pred
#     print(Y)
#     print(np.exp(y_pred))
# evaluation(X,Y,'original')
# evaluation(exp_x,exp_y,'quadratic')
# evaluation(exp_x,Y,'quadratic')
# evaluation(X,exp_y,'quadratic')
# evaluation(exp_x,ln_y,'quadratic')
# evaluation(ln_X,ln_y,'quadratic')
# evaluation(ln_X,Y,'quadratic')
# evaluation(X,ln_y,'quadratic_positive')
# np.exp(ln_y)
# pr.coef_
# pr.intercept_

y_pred = predict(test_X,np.log(test_y),'quadratic_positivemodel.pkl')
# ori_y_pred = predict(test_X,test_y,'originalmodel.pkl')


In [342]:
test_df['quad_pred'] = np.exp(y_pred)
test_df['accuracy'] = test_df['accuracy']/100
test_df = test_df.reindex(columns = ['ap2','yearmonth','bias','accuracy','quad_pred'])
# print(test_df.reset_index())
# test_df['error'] = test_df['accuracy'] - test_df['y_pred']
# print(test_df)


In [343]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

